For the longest time, I've been wanting to do some analysis using facebook graph API. One of the domains which really excites me, in terms of analysing data and deriving conclusions from it, is studying data related to human behaviour. And what better source of such information do we have other than facebook?  I mean, it has more than [2 billion monthly active users](https://en.wikipedia.org/wiki/List_of_virtual_communities_with_more_than_100_million_active_users). It is a cesspool of narcissicm where people share their important moments.

I can definitely acknowledge that the dopamine rush which facebook provides is a bit "dangerous", its almost like a drug, and is prone to addiction like any other feel good activity. But when used in moderation, it provides benefits like any other drugs - a place to express yourself and share stories in a world which is becoming less social by the minute.

However, this raises the important question of privacy, and facebook has recognized the importance of that by modifying its API over the years. While earlier it was easy to access users' public information by using the API, now it requires the user to authenticate before you can access even the public information. Unfortunately, that makes it difficult for people like me who are looking to use other people's data to derive generalized insights. So, until I'm able to convince a considerable number of people to give their precious data, I decided to work on what I had - my own data. I decided to analyze my posts over the years to see if I could find some patterns.

To start with, we try to answer the question : what are the features which describe a facebook post? I came up with the following answers:

* Number of likes
* Length of the post
* Time of the post (Month, Day of week, Time of day)
* Content of the post 

Next, use facebook's graph API to get the above data, and write it to a csv file. If you want to do this for your own facebook statuses, you'll need an FB API user access token (I generated mine via the [Graph API Explorer](https://developers.facebook.com/tools/explorer/145634995501895/)). You'll also need a [Facebook developer account](https://developers.facebook.com/) for this.

```python
import facebook, datetime
from textblob import TextBlob
import csv
writer = csv.writer(open('data-statuses.csv', 'wb'))
writer.writerow(['polarity', 'subjectivity', 'num_reactions', 'length', 'num_words', 'month', 'dow', 'tod'])
graph = facebook.GraphAPI(access_token='youraccesstoken', version='2.8')
posts = graph.get_all_connections(id='me', connection_name='posts')
for post in posts:
    try:
        post_id = post['id']
        status = post['message']

        time_string = post['created_time'][:-5]
        time = datetime.datetime.strptime(time_string, '%Y-%m-%dT%H:%M:%S')
        time = post['time']

        blob = TextBlob(status)
        polarity, subjectivity = blob.sentiment
        length = len(status)
        num_words = len(status.split(" "))
        post_reactions = graph.get_all_connections(id=post_id, connection_name="reactions")
        num_reactions = sum(1 for _ in post_reactions)

        month = time.month
        dow = time.weekday()
        tod = time.hour

        writer.writerow([polarity, subjectivity, num_reactions, length, num_words, month, dow, tod])
    except:
        continue

```

Now, lets start the analysis. We use pandas to load the data in the csv file to a dataframe.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('data-statuses.csv')
print df.head()
print df.shape

   polarity  subjectivity  num_reactions  length  num_words  month  dow  tod
0  0.125000      0.458333             18     312         59      7    4   20
1 -0.245833      0.320833              1     107         18      7    3   19
2 -0.400000      0.400000             50      56          7      7    3   12
3  0.000000      0.000000              0       6          1      6    6   16
4  0.000000      0.000000              3       1          1      6    0   18
(703, 8)
